In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
Dados = pd.read_csv (r'../input/wineuci/Wine.csv', delimiter=',', encoding = "ISO-8859-1", 
                     names = ('class','Alcohol','Malic acid','Ash','Alcalinity of ash',
                                'Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols',
                                'Proanthocyanins','Color intensity','Hue','OD','Proline'))   
Dados

In [ ]:
# Checking nulls
round(100 * (Dados.isnull().sum()/len(Dados)),2)

In [ ]:
Dados.describe()

In [ ]:
Dados.groupby('class')['class'].count()

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(20, 20))
i = 0
j = 0
for vars in Dados.columns :
    sns.histplot(data = Dados, x = vars, bins = 20, ax = axes[i,j])
    j = j + 1
    if j == 4 :
        i = i + 1
        j = 0

In [ ]:
# Correlation Matrix
plt.figure(figsize=(18, 8))
sns.heatmap(Dados.corr(), vmin=-1, vmax=1, annot=True, cmap='vlag') # cmap='BrBG'
plt.title('Correlation Map', fontdict={'fontsize':12}, pad=12);

In [ ]:
sns.histplot(data = Dados, x = 'Flavanoids', bins = 20, hue = 'class')

In [ ]:
sns.pairplot(data = Dados)

In [ ]:
sns.jointplot(data=Dados, x='Flavanoids', y='Alcalinity of ash', hue = 'class')

In [ ]:
sns.jointplot(data=Dados, x='Flavanoids', y='Total phenols', hue = 'class')

In [ ]:
sns.jointplot(data=Dados, x='Flavanoids', y='OD', hue = 'class')

In [ ]:
sns.jointplot(data=Dados, x='Color intensity', y='Proline', hue = 'class')

In [ ]:
Dados.columns

In [ ]:
DadosX = Dados.iloc[:,1:14]
DadosY = Dados.iloc[:,0]

In [ ]:
DadosX.columns

In [ ]:
DadosY

In [ ]:
# Splitting data

X_train, X_test, Y_train, Y_test = train_test_split(DadosX, DadosY,random_state=101, test_size=0.3)

In [ ]:
# Multi MLP Generator
training_accuracy = []
testing_accuracy = []
Layer1 = range(10,90,20)
Layer2 = range(10,90,20)
LayersComb = len(Layer1)*len(Layer2)
Step=0
score=0

for i in Layer1 :
  for j in Layer2 :
    mlp = MLPClassifier(hidden_layer_sizes=(i,j),activation="logistic" ,random_state=1, max_iter=2000).fit(X_train, Y_train)
    Y_pred_train = mlp.predict(X_train).round()
    training_accuracy.append(accuracy_score(Y_train, Y_pred_train))
    Y_pred_test = mlp.predict(X_test).round()
    acc_score = accuracy_score(Y_test,Y_pred_test)
    testing_accuracy.append(acc_score)
    Step = Step + 1
    if score < acc_score:
        score = acc_score
        best_Layer1 = i
        best_Layer2 = j
        best_Step = Step
    
    print('Step ', Step, ' of ', LayersComb, ' Layer1: ', i, ' Layer2: ', j, ' Accuracy', acc_score.round(4))    
    print('Best Accuracy Score', score.round(4), ' Best Layer1: ', best_Layer1, ' Best Layer2: ', best_Layer2, ' do Step:', best_Step)

In [ ]:
# 2 Layers
sns.lineplot(data=[training_accuracy,testing_accuracy])

In [ ]:
# Selected Model - 2 Layers
mlp_selected = MLPClassifier(hidden_layer_sizes=(30,10),activation="logistic" ,random_state=1, max_iter=2000)
mlp_selected.fit(X_train, Y_train)

In [ ]:
Y_pred_train = mlp_selected.predict(X_train).round()
training_accuracy.append(accuracy_score(Y_train, Y_pred_train))
Y_pred_test = mlp_selected.predict(X_test).round()
acc_score = accuracy_score(Y_test,Y_pred_test)
testing_accuracy.append(acc_score)

In [ ]:
# Confusion Matrix for Selected Model - 2 Layers
cm=confusion_matrix(Y_test,Y_pred_test)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:1','Predicted:2','Predicted:3'],index=['Actual:1','Actual:2','Actual:3'])
plt.figure(figsize = (8,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu");

In [ ]:
sns.lineplot(data=mlp_selected.loss_curve_)

In [ ]:
print(classification_report(Y_test,Y_pred_test))

In [ ]:
mlp_selected.predict_proba(X_test)